In [1]:
!pip install pandas openpyxl xlrd


Defaulting to user installation because normal site-packages is not writeable


In [23]:
#FOR 585,587,AG POS
import pandas as pd

# --- Step 1: Read Excel and skip top rows ---
file_path = r"E:\OrderProcssing\data\Pushpam (PCB)\Internal Order Format 14-July-2025-Silver (CZ).xlsx"
df = pd.read_excel(file_path, skiprows=1)

# --- Step 2: Assign meaningful column names ---
df.columns = [
    "SrNo",
    "OrderGroup",
    "ItemPoNo",
    "StyleCode",
    "Metal",
    "ItemSize",
    "Purity",
    "Nos",
    "OrderQty",
    "GrossWt",
    "NetWt",
    "StoneWt",
    "StonePcs",
    "CustomerProductionInstruction",
    "Category"   # Special remark coln in PO
]

# --- Step 3: Map Metal to Tone ---
tone_map = {
    "AG925": "",
    "G585W": "W",
    "G585P": "P",
    "G585Y": "Y",
    "G585WZ": "W",
    "G585YZ": "Y",
    "G585PZ": "P",
    "G585W-NI1811-RHC": "W",
    "G585Y-C143GR": "Y",
    "G585W-NPF301": "W",
    "G585P-C145N": "P",
    "G587Y": "Y"
}
df['Tone'] = df['Metal'].map(tone_map).fillna("")

# --- Step 4: Insert Tone after Metal ---
cols = df.columns.tolist()
metal_index = cols.index("Metal")
cols.remove("Tone")
cols.insert(metal_index + 1, "Tone")
df = df[cols]

# --- Step 5: Add new columns after ItemPoNo ---
# Get index of ItemPoNo
itempo_index = df.columns.get_loc("ItemPoNo")

# Create new columns with default blank values
df.insert(itempo_index + 1, "ItemRefNo", "")
df.insert(itempo_index + 2, "StockType", "")

# Take user input for Priority
priority_value = input("Enter Priority value for all rows: ")   #TMP SMP REPSMP REVSMP SDPL CUSCAD MSP
df.insert(itempo_index + 3, "Priority", priority_value)

df.insert(itempo_index + 4, "MakeType", "")

# --- Step 6: Select and reorder final columns ---
selected_columns = [
    'SrNo', 'StyleCode', 'ItemSize', 'OrderQty', 'Metal', 'Tone',
    'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority', 'MakeType',
    'CustomerProductionInstruction', 'OrderGroup'
]

df_selected = df[selected_columns].copy()
# Make NaN values in CustomerProductionInstruction blank
df_selected['CustomerProductionInstruction'] = df_selected['CustomerProductionInstruction'].fillna("")
# --- Step 7: Preview ---
# --- Add SpecialRemarks column after CustomerProductionInstruction ---
cpi_index = df_selected.columns.get_loc("CustomerProductionInstruction")
df_selected.insert(
    cpi_index + 1,
    "SpecialRemarks",
    "CUSTOMER-PCB, PO#PCB - " + df_selected['ItemPoNo'].astype(str) + ",NEW SAMPLE ORDER, DIAMOND GRADE-CZ"
)

special_index = df_selected.columns.get_loc("SpecialRemarks")

# Mapping function
def map_design_instruction(tone):
    if tone == "W":
        return "WHITE RODIUM"
    else:  # Y, PT, or blank
        return "NO RODIUM"

# Insert column after SpecialRemarks
df_selected.insert(
    special_index + 1,
    "DesignProductionInstruction",
    df_selected['Tone'].apply(map_design_instruction)
)

# --- Add StampInstruction column after DesignProductionInstruction ---
dpi_index = df_selected.columns.get_loc("DesignProductionInstruction")

df_selected.insert(
    dpi_index + 1,
    "StampInstruction",
    "LT STAMP (LOGO) + " + df_selected['Metal'].astype(str)
)

# --- Add new columns after OrderGroup ---
ordergroup_index = df_selected.columns.get_loc("OrderGroup")

# Take user input for SKUNo
skuno_value = input("Enter SKUNo value for all rows: ")

# Insert columns one by one after OrderGroup
df_selected.insert(ordergroup_index + 1, "Certificate", "")
df_selected.insert(ordergroup_index + 2, "SKUNo", skuno_value)
df_selected.insert(ordergroup_index + 3, "Basestoneminwt", "")
df_selected.insert(ordergroup_index + 4, "Basestonemaxwt", "")
df_selected.insert(ordergroup_index + 5, "Basemetalminwt", "")
df_selected.insert(ordergroup_index + 6, "Basemetalmaxwt", "")
df_selected.insert(ordergroup_index + 7, "Productiondeliverydate", "")
df_selected.insert(ordergroup_index + 8, "Expecteddeliverydate", "")

df_selected.to_csv(r"C:\Users\Admin\Desktop\PCB_Cleaned.csv", index=False)
df_selected.head()


Enter Priority value for all rows:  TMP
Enter SKUNo value for all rows:  23456


,SrNo,StyleCode,ItemSize,OrderQty,Metal,Tone,ItemPoNo,ItemRefNo,StockType,Priority,...,StampInstruction,OrderGroup,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate
0,1,RG0003410F,EU54,1,AG925,,PCB - Eppi (14.07.25),,,TMP,...,LT STAMP (LOGO) + AG925,PCB,,23456,,,,,,
1,2,RG0003447A,EU54,1,AG925,,PCB - Eppi (14.07.25),,,TMP,...,LT STAMP (LOGO) + AG925,PCB,,23456,,,,,,
2,3,RG0003480A,EU54,1,AG925,,PCB - Eppi (14.07.25),,,TMP,...,LT STAMP (LOGO) + AG925,PCB,,23456,,,,,,
3,4,RG0003479A,EU54,1,AG925,,PCB - Eppi (14.07.25),,,TMP,...,LT STAMP (LOGO) + AG925,PCB,,23456,,,,,,
4,5,RG0003445D,EU54,1,AG925,,PCB - Eppi (14.07.25),,,TMP,...,LT STAMP (LOGO) + AG925,PCB,,23456,,,,,,
